In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [21]:
# Import data
weather_data_df = pd.read_csv('../WeatherPy/output_data/weather_df.csv')
weather_data_df

,City,Lat,Lng,Maxtemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Airai,-8.9266,125.4092,60.26,90,91,3.13,TL,1651329814
1,Kirakira,-10.4544,161.9205,79.93,81,100,0.58,SB,1651329897
2,Port Alfred,-33.5906,26.8910,67.17,66,2,7.00,ZA,1651329657
3,Sarakhs,36.5449,61.1577,96.94,19,75,4.61,IR,1651329897
4,Vaini,-21.2000,-175.2000,77.16,94,100,10.36,TO,1651329801
...,...,...,...,...,...,...,...,...,...
562,Flin Flon,54.7682,-101.8650,40.84,75,75,4.61,CA,1651329871
563,Krasnoarmeyskoye,52.7182,50.0322,55.17,53,73,6.20,RU,1651330146
564,Rurrenabaque,-14.4413,-67.5278,81.23,65,85,3.67,BO,1651330147
565,Vestmannaeyjar,63.4427,-20.2734,47.39,76,100,16.11,IS,1651329903


 ### Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight.
- Add Heatmap layer to map.

In [23]:
# Configure Google Maps API
gmaps.configure(api_key=g_key)

# Locations
locations = weather_data_df[['Lat', 'Lng']]

# Humidity
humidity = weather_data_df['Humidity']

# Heat map plot
fig = gmaps.figure(center=(0, 0), zoom_level=2)
# Create heat map layer
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100, point_radius=5)

# Add heatmap layer to map
fig.add_layer(heatmap_layer)
fig 

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [42]:
# Narrow down the cities to fit weather conditions
ideal_temp = weather_data_df[(weather_data_df['Maxtemp'] < 85)&  (weather_data_df['Maxtemp'] >75)]
ideal_cloudiness = ideal_temp[ideal_temp['Cloudiness'] < 20]
ideal_windspeed = ideal_cloudiness[ideal_cloudiness['Wind Speed'] < 10]

# Drop the rows with null values and index column after resetting the index
ideal_vacation_spots = ideal_windspeed[ideal_windspeed['Humidity'] < 60].dropna()
ideal_vacation_spots = ideal_vacation_spots.reset_index()
del ideal_vacation_spots['index']
ideal_vacation_spots

,City,Lat,Lng,Maxtemp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Evans,33.5337,-82.1307,75.25,56,0,8.05,US,1651329929
1,Esmeraldas,-19.7625,-44.3139,80.71,37,0,6.64,BR,1651329966
2,Lázaro Cárdenas,17.9583,-102.2000,79.99,54,1,2.73,MX,1651329970
3,Chiredzi,-21.0500,31.6667,80.28,37,0,2.04,ZW,1651329976
4,Bahía Honda,22.9036,-83.1592,81.16,54,14,9.48,CU,1651329978
5,Caravelas,-17.7125,-39.2481,83.50,57,6,7.40,BR,1651329886
6,Tomatlán,19.9333,-105.2500,79.43,30,7,1.86,MX,1651329859
7,Yangi Marg`ilon,40.4272,71.7189,81.32,29,0,2.30,UZ,1651330022
8,Hwange,-18.3693,26.5019,82.69,27,0,4.63,ZW,1651330036
9,Coahuayana Viejo,18.7333,-103.6833,82.69,33,1,2.59,MX,1651329927
